In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
train.head()

In [ ]:
train.describe

In [ ]:
train.shape

In [ ]:
train.isnull().sum()

In [ ]:
sns.heatmap(train.isnull(), cbar = False, cmap = 'viridis')

In [ ]:
sns.heatmap(test.isnull(), cbar = False, cmap = 'viridis') #test

In [ ]:
train.Age.isnull().sum()/train.shape[0]*100  #calculating what percentage of the Ages are null values

In [ ]:
ax = train.Age.hist(bins = 30, density = True, stacked = True, color = 'magenta', alpha = 0.7, figsize = (16, 5))
train.Age.plot(kind = 'density')
ax.set_xlabel('Age')
plt.show()

In [ ]:
#how age affected survival

Survived = 'Survived'
not_survived = 'did not survive'

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (10, 4))
women = train[train['Sex'] == 'female']
men = train[train['Sex'] == 'male']

ax = sns.distplot(women[women[Survived]==1].Age.dropna(), bins = 18, label = Survived, ax = axes[0], kde = False)
ax = sns.distplot(women[women[Survived]==0].Age.dropna(), bins = 40, label = not_survived, ax = axes[0], kde = False)
ax.legend()
ax.set_title('Female Survival')

ax = sns.distplot(men[men[Survived]==1].Age.dropna(), bins = 18, label = Survived, ax = axes[1], kde = False)
ax = sns.distplot(men[men[Survived]==0].Age.dropna(), bins = 40, label = not_survived, ax = axes[1], kde = False)
ax.legend()
ax.set_title('Male Survival')

In [ ]:
train.Sex.value_counts()

In [ ]:
sns.catplot(x = 'Pclass', y = 'Age', data = train, kind = 'box')

In [ ]:
sns.catplot(x = 'Pclass', y = 'Fare', data = train, kind = 'box')

In [ ]:
train[train['Pclass'] == 1]['Age'].mean() #mean age of 1st class passengers

In [ ]:
train[train['Pclass'] == 2]['Age'].mean()

In [ ]:
train[train['Pclass'] == 3]['Age'].mean()

In [ ]:
print(test[test['Pclass'] == 1]['Age'].mean())
print(test[test['Pclass'] == 2]['Age'].mean())
print(test[test['Pclass'] == 3]['Age'].mean())

In [ ]:
#fill missing ages with mean of Pclass Age

def impute_Age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return train[train['Pclass'] == 1]['Age'].mean()
        elif Pclass == 2:
             return train[train['Pclass'] == 2]['Age'].mean()
        elif Pclass == 3:
            return train[train['Pclass'] == 2]['Age'].mean()
        
    else:
        return Age

In [ ]:
train['Age'] = train[['Age', 'Pclass']].apply(impute_Age, axis = 1)

In [ ]:
sns.heatmap(train.isnull(), cbar = False, cmap = 'viridis')

In [ ]:
test['Age'] = test[['Age', 'Pclass']].apply(impute_Age, axis = 1)

In [ ]:
sns.heatmap(test.isnull(), cbar = False, cmap = 'viridis')

In [ ]:
f = sns.FacetGrid(train, row = 'Embarked', height = 2.5, aspect = 3)
f.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', order = None, hue_order = None)
f.add_legend()

In [ ]:
train.Embarked.isnull().sum()

In [ ]:
train['Embarked'].value_counts()

In [ ]:
common_value = 'S'
train['Embarked'].fillna(common_value, inplace = True)

In [ ]:
train.Embarked.isnull().sum()

In [ ]:
test.Fare.isnull().sum()

In [ ]:
fill_fare = test.Fare.mean()

In [ ]:
test['Fare'].fillna(fill_fare, inplace = True)

In [ ]:
test.Fare.isnull().sum()

In [ ]:
#drop cabin and ticket column

train.drop(labels = ['Cabin', 'Ticket', 'Name', 'PassengerId'], inplace = True, axis = 1)
test.drop(labels = ['Cabin','Name', 'Ticket'], inplace = True, axis = 1)

In [ ]:
sns.heatmap(train.isnull(), cbar = False, cmap = 'viridis')

In [ ]:
sns.heatmap(test.isnull(), cbar = False, cmap = 'viridis')

In [ ]:
#feature transformation, categorical values into integers

train.head()

In [ ]:
train.info()

In [ ]:
train.Fare = train.Fare.astype('int')
train.Age = train.Age.astype('int')
train.info()

In [ ]:
test.Fare = train.Fare.astype('int')
test.Age = train.Age.astype('int')
test.info()

In [ ]:
Gender = {'male': 0, 'female': 1}
train.Sex = train.Sex.map(Gender)

Port = {'S': 0, 'C': 1, 'Q': 2}
train.Embarked = train.Embarked.map(Port)
train.head()

In [ ]:
Gender = {'male': 0, 'female': 1}
test.Sex = test.Sex.map(Gender)

Port = {'S': 0, 'C': 1, 'Q': 2}
test.Embarked = test.Embarked.map(Port)
test.head()

In [ ]:
test.info()

**Logistic Regression Model**

In [ ]:
X = train.drop('Survived', axis = 1)
y = train.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
X_train.shape

In [ ]:
model = LogisticRegression(solver = 'lbfgs', max_iter = 400)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [ ]:
model.score(X_test, y_test)

**Test**

In [ ]:
test.head()

In [ ]:
wanted_test_columns = X_train.columns
wanted_test_columns

In [ ]:
predictions = model.predict(test[wanted_test_columns])

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = test['PassengerId']
submission['Survived'] = predictions
submission

In [ ]:
submission.to_csv('../logisticregression_submission.csv', index=False)

In [ ]:
test_submission = pd.read_csv('../logisticregression_submission.csv')
test_submission